In [ ]:
%pip install scipy
%pip install ripser gudhi
%pip install scikit-tda
%pip install numpy
%pip install tqdm
%pip install networkx

## 1. Preprocessing

In [ ]:
from src.preprocessing.preprocessing import *


VR persistence diagram with Ripser

In [1]:
import os
from src.shape.shape import *
# Load preprocessed data

data_path = "data/preprocessed/"
shape_paths = []
for file in os.listdir(data_path):
    if file.endswith(".mat"):
        shape_paths.append(data_path + file)

print(f"Found {len(shape_paths)} shapes... {shape_paths[0], shape_paths[1]}, ...")
shapes = []
for f in shape_paths:
    shapes.append(Shape(f))
    
    

Found 148 shapes... ('data/preprocessed/pre_geodesic_gorilla8.mat', 'data/preprocessed/pre_geodesic_victoria1.mat'), ...
